In [1]:
import requests
import json
import os
import time
import hashlib

from io import BytesIO
from functools import wraps
from pathlib import Path
from enum import Enum

import imagehash
from PIL import Image

# Config

In [52]:
API_ROUTE = "https://api.twitter.com/2/"
STREAM_ROUTE = "tweets/search/stream"
RULES_ROUTE = "tweets/search/stream/rules"
FULL_SEARCH_ROUTE = "tweets/search/all"
ROOT_FOLDER = r"/home/tyra/Documents/Collectes"
# ROOT_FOLDER = r"C:\Users\Orion\Documents\OutputTweets"
CREDENTIALS_FILES = r"/home/tyra/Documents/CERES/credentials_pro.json"
# CREDENTIALS_FILES = r"C:\Users\Orion\Documents\Projets\CERES\credentials_pro.json"
# maximum size of the collect in octets
MAX_SIZE = 60073741824
TOKEN = generate_token()
params = {
    "tweet.fields": "public_metrics,referenced_tweets",
    "expansions": "author_id,in_reply_to_user_id,attachments.media_keys,attachments.poll_ids",
    "media.fields": "url,alt_text",
    "user.fields": "id,verified,description",
    "poll.fields": "duration_minutes,end_datetime"
}

#### Load Token

In [5]:
res = requests.get("https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/1920px-Image_created_with_a_mobile_phone.png")

In [6]:
buffer = BytesIO(res.content)

In [17]:
class A:
    pass

In [25]:
a = A()
type(a).__name__

'A'

In [16]:
sha1Hash = hashlib.sha1(buffer.read())

In [11]:
sha1Hash.hexdigest()

'4a2e56b769afdb3e15176f98d7e5375a62bb03d0'

In [30]:
def generate_token():
    with open(CREDENTIALS_FILES, 'r') as f:
        return f"Bearer {json.load(f)['token']}"

In [36]:
s = requests.Session()
s.headers.update({"Authorization": TOKEN})

In [37]:
class Media(Enum):
    PHOTO = 1
    VIDEO = 2
    GIF = 3
    
    def __eq__(self, other):
        if isinstance(other, str):
            return self.name.lower() == other

In [38]:
def timeit(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        res = f(*args, **kwargs)
        print(f"{f.__name__} executed in {time.time() - t1}")
        return res
    return wrapper

In [ ]:
# test_handle_tweet
tweet = """
{
    "data": {
        "attachments": {
            "media_keys": [
                "3_1437787186561228800"
            ]
        },
        "author_id": "21313364",
        "entities": {
            "urls": [
                {
                    "start": 129,
                    "end": 152,
                    "url": "https://t.co/1kUyRMrCmS",
                    "expanded_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086",
                    "display_url": "nouvelobs.com/droits-des-fem…",
                    "images": [
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=orig",
                            "width": 1024,
                            "height": 512
                        },
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=150x150",
                            "width": 150,
                            "height": 150
                        }
                    ],
                    "status": 200,
                    "title": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois",
                    "description": "Le Parlement a voté fin juin la procréation médicalement assistée pour toutes les femmes, qu’elles soient hétérosexuelles, homosexuelles ou monoparentales. Une enveloppe de 8 millions d’euros doit permettre de répondre à l’afflux de demandes.",
                    "unwound_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086"
                },
                {
                    "start": 153,
                    "end": 176,
                    "url": "https://t.co/1uGLGEP38c",
                    "expanded_url": "https://twitter.com/lobs/status/1437787188868091906/photo/1",
                    "display_url": "pic.twitter.com/1uGLGEP38c"
                }
            ]
        },
        "id": "1437787188868091906",
        "organic_metrics": {},
        "text": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois https://t.co/1kUyRMrCmS https://t.co/1uGLGEP38c"
    },
    "includes": {
        "media": [
            {
                "media_key": "3_1437787186561228800",
                "type": "photo",
                "url": "https://pbs.twimg.com/media/E_QL1BWXIAAEEPV.jpg"
            }
        ],
        "users": [
            {
                "id": "21313364",
                "name": "L'Obs",
                "username": "lobs"
            }
        ]
    },
    "matching_rules": [
        {
            "id": 1437707103754506241,
            "tag": "PMA"
        }
    ]
}"""
handle_tweet(json.loads(tweet))


In [39]:
def handle_media(tweet):
    # check if there are some media in the tweet
    if not tweet.get('includes', {}).get('media', False):
        return
    # extract the media
    media = tweet['includes']['media']
    # TODO: use concurrent to download simultanously or even in fire and forget
    for medium in media:
        if medium['type'] == Media.PHOTO:
            media_url = medium['url']
        else:
            print(f"unhandled media type currently: {medium['type']}")
            continue
        # if we already have stored this media (avoid to redownload in case of retweet)
        if medium['media_key'] in [x.split('.')[0] for x in os.listdir(os.path.join(ROOT_FOLDER, 'media'))]:
            print("Media already downloaded")
            pass
        else:
            download_media(**medium)

In [40]:
def compute_signature(buffer):
    img = Image.open(BytesIO(buffer))
    return str(imagehash.average_hash(img))

In [41]:
def download_media(media_key=None, url=None, **kwargs):
    if not media_key or not url:
        raise ValueError("Missing field when trying to save media")
    file_type = url.split('.')[-1]
    file_name = f"{media_key}.{file_type}"
    
    # download the file
    try:
        res = requests.get(url)
        print("downloading media")
    except requests.RequestException:
        raise ValueError(f"There was an error when downloading the media with following url: {url}, please check your connection or url")
    
    # calculate signature of content, if this signature already exists then just increment the number of 
    buffer = res.content
    signature = compute_signature(buffer)
    if signature in os.listdir(os.path.join(ROOT_FOLDER, 'media')):
        
        # lets find the id of the identical file, already saved as an empty file under media/<signature>/id
        identical_media_key = os.listdir(os.path.join(ROOT_FOLDER, 'media', signature))[0]
        
        print(f"a media with the same signature already exists: {identical_media_key}")
        
        # we need to save an empty file to know how to find the existing image from the media_key
        with open(os.path.join(ROOT_FOLDER, 'media', f"{media_key}.{identical_media_key}"), 'w') as f:
            f.write('')
        return
    else:
        path = os.path.join(ROOT_FOLDER, 'media', signature)
        Path(path).mkdir(exist_ok=True, parents=True)
        with open(os.path.join(path, media_key), 'w') as f:
            # create an empty file named with the id so we know this signature = this id
            f.write('')
    # save to disk
    with open(os.path.join(ROOT_FOLDER, 'media', file_name), 'wb') as f:
        f.write(res.content)

In [42]:
@timeit
def handle_tweet(tweet):
    id = tweet['data']['id']
    
    # filter tags:
    tags = list(set([r['tag'] for r in tweet["matching_rules"]]))
    
    # save media:
    handle_media(tweet)
    
    # save collected tweet in every tag
    for tag in tags:
        # create directory if not exist
        directory = Path(os.path.join(ROOT_FOLDER), tag)
        directory.mkdir(parents=True, exist_ok=True)
        with open(os.path.join(directory, f"{id}.json"), 'w', encoding='utf-8') as f:
            json.dump(tweet, f, indent=4, ensure_ascii=False)

In [43]:
def get_folder_size(path):
    return sum(f.stat().st_size for f in Path(path).glob('**/*') if f.is_file())

In [44]:
@timeit
def has_free_space():
    if not MAX_SIZE:
        return True
    if get_folder_size(ROOT_FOLDER) < MAX_SIZE:
        return True
    raise OSError("The maxsize of the storage directory has been reached")

In [45]:
def get_rules():
    """
    Allow to get all rules currently active for the collect
    """
    rules = s.get(API_ROUTE + RULES_ROUTE)
    return rules.json()

In [46]:
def get_tags_from_rules(rules):
    if 'data' not in rules:
        raise ValueError('You seem to have no rules configured yet, please make sure to create some')
    return list(set([r['tag'] for r in rules['data']]))

In [47]:
def init_storages(folders=[]):
    """
    Ensure folders are properly created at the begining of the collect
    """
    for f in folders:
        directory = Path(os.path.join(ROOT_FOLDER, f))
        directory.mkdir(parents=True, exist_ok=True)

In [68]:
try:
    res = requests.get("https://httpbin.org/status/400")
    res.raise_for_status()
except Exception as e:
    print(e.response.status_code)

400


In [53]:
def collect():
    # first ensure all folders needed are created
    init_storages(['users', 'media', *get_tags_from_rules(get_rules())])
    
    # then connect to the stream
    with s.get(API_ROUTE + STREAM_ROUTE, params=params, stream=True, timeout=5000) as resp:
        if resp.status_code != 200:
            print(f"error {resp.status_code}")
            print(resp.content)
        for line in resp.iter_lines():
            if line and has_free_space():
                data = json.loads(line.decode("utf-8"))
                if 'data' not in data:
                    print(data)
                print(data['data']['text'])
                print(data['data']['id'])
                handle_tweet(data)
            else:
                print("waiting for new tweets")

In [54]:
collect()

has_free_space executed in 0.03391551971435547
Ne confondez pas victimes et agresseurs ! Tout cela pour faire passer la pilule car vous n'avez pas fait votre boulot.
La lèpre du foot... C'est contagieux, la lèpre. Tout le Sud est infecté.
Et le furoncle de la police vous connaissez @Prefet34 ? https://t.co/ZTsIsAEAEd
1441070535371329538
handle_tweet executed in 0.001369476318359375
has_free_space executed in 0.014328718185424805
@BFMTV Mr Burns qui couche avec Lisa c'est pas bien... https://t.co/whFq4POZHI
1441070523119595522
downloading media
handle_tweet executed in 0.6483962535858154
has_free_space executed in 0.010313749313354492
@afrodisiss Toi et zemmour
1441070535983652864
handle_tweet executed in 0.0005428791046142578
has_free_space executed in 0.0328369140625
@ericcoquerel @JLMelenchon les programmes...... https://t.co/foo9TkH1DB
1441070538982576129
downloading media
handle_tweet executed in 0.28959226608276367
has_free_space executed in 0.01653575897216797
@Cemil @JLMelenchon

waiting for new tweets
has_free_space executed in 0.03254842758178711
@TeamProgressist Elle sera bientôt votre pire cauchemar
1441070605898588166
handle_tweet executed in 0.0012388229370117188
has_free_space executed in 0.015387535095214844
@MLeTrocquer_ @JLMelenchon Je pose ca la:

Au fait les #JeSoutiensMelenchon vous connaissez cette histoire de harcèlement dans le parti des insoumis? La vérité sur #melanchon ⤵️

https://t.co/dZyuwbO9Og
1441070605961400320
handle_tweet executed in 0.0006418228149414062
has_free_space executed in 0.03191685676574707
@PoliceRealites Et oui la liberté d’expression pour Zemmour el la même pour un artiste
1441070612076695555
handle_tweet executed in 0.0009961128234863281
has_free_space executed in 0.03190302848815918
Qu'est-ce qui me répugne le plus entre ce débat et le rosé pamplemousse ?  🤔 https://t.co/EfirV6Rl7B
1441070616019410948
handle_tweet executed in 0.0009429454803466797
has_free_space executed in 0.013529062271118164
@nidr_f @ducasse_paula @B

has_free_space executed in 0.03328394889831543
@RMCsport Logique
1441070678606819333
handle_tweet executed in 0.0009138584136962891
has_free_space executed in 0.013869762420654297
En voila un gouvernement responsable pas comme @EmmanuelMacron and co https://t.co/4SobExOCjy
1441070676249620485
handle_tweet executed in 0.0005567073822021484
has_free_space executed in 0.010190010070800781
@BFMTV En tout cas je ne veux pas aller au paradis. Je n'y connaîtrais sûrement personne ! 😂
1441070676308369418
handle_tweet executed in 0.0005333423614501953
has_free_space executed in 0.03299736976623535
Mieux qu’une télé-réalité réalité ce soir c’est soirée pop-corn devant melenchon  VS zemmour #BFMTV #ZemmourVsMelenchon
1441070679869198344
handle_tweet executed in 0.0011544227600097656
has_free_space executed in 0.014154195785522461
Je regarde un ancien débat melenchon zemmour pour me préparer à celui de ce soir
1441070681337249793
handle_tweet executed in 0.0006296634674072266
has_free_space execut

downloading media
downloading media
downloading media
handle_tweet executed in 2.2784764766693115
has_free_space executed in 0.024411439895629883
Ptdrr carrément un hashtag de soutien pour Zemmour, oh c juste un débat ils vont pas se taper sur la gueule
1441070734193958913
handle_tweet executed in 0.0006256103515625
has_free_space executed in 0.011516094207763672
@BFMTV C'est pas fait pour les chochottes 👌
1441070736978980864
handle_tweet executed in 0.0005500316619873047
has_free_space executed in 0.010554313659667969
@TeamProgressist Elle n'est rien et ne représente rien mais dans le bal des faux culs, elle a sa place
1441070737587118081
handle_tweet executed in 0.00025773048400878906
has_free_space executed in 0.010244607925415039
tellement genant cette surmédiatisation de ce débat de merde entre deux merdes https://t.co/yMOvUQ3Pve
1441070742054047747
handle_tweet executed in 0.0005724430084228516
has_free_space executed in 0.03492617607116699
@SuisNadege @EricNaulleau Il trompe sa 

has_free_space executed in 0.030076980590820312
@superhenrytran Il y en a qui se sont faits piéger avec le débat EZ/JLM : c'est V. Pécresse/G. Darmanin, qui débattent, à la même heure sur France 2. Et inévitablement, les audiences seront scrutées et comparées. Je crains la cata d'un côté, mais je ne dis pas lequel.
1441070799570440196
handle_tweet executed in 0.0007808208465576172
has_free_space executed in 0.03448772430419922
@F_Desouche Parce que les Lepen depuis 40 ans ..
1441070804884721666
handle_tweet executed in 0.0008020401000976562
has_free_space executed in 0.027367115020751953
@BFMTV Ça va saigner, ça va gicler, lequel va dévoiler son pire côté en 1er ?
1441070802296840192
handle_tweet executed in 0.0007162094116210938
has_free_space executed in 0.0191497802734375
@PieLimpide @christine_kelly @ZemmourEric Qu'est. Ce qui vous chagrine...??  Juste que vous n'avez rien à dire et aucune pensée originale.....coucouche panier !
1441070805232783365
handle_tweet executed in 0.000788

...

has_free_space executed in 0.12920141220092773
@BastienParisot @JLMelenchon Ah bon ! Pourtant Mélenchon est aussi identitaire que Zemmour, normale c’est aussi une extrémiste, de gauche.
1441085669925801990
handle_tweet executed in 0.0006549358367919922
has_free_space executed in 0.12757349014282227
@BFMTV Encore un mensonge. On creusera après. Tout ça pour faire gober aux français que le vaccin est inoffensif du nourrisson au jeune adulte... Quel goût de la perversion, de la malhonnêteté...
1441085667769913346
handle_tweet executed in 0.000568389892578125
has_free_space executed in 0.15196681022644043
@kindelsen @Traineur2 @MurielCousin @MARTINERAOUX @sandrousseau Le pen vote de la même façon que les rouges et les verts.
Quant aux affaires,, avec toutes les casseroles que se trimbalent les pastèques, tu ferais mieux de fermer ce qui te sert de bouche et qui pue comme un cul
1441085671028842499
handle_tweet executed in 0.0006442070007324219
has_free_space executed in 0.1562056541442871


has_free_space executed in 0.16225814819335938
@RMCsport Bravo, merci pour l'info, mais on s'en fout
1441085739358253072
handle_tweet executed in 0.0006983280181884766
has_free_space executed in 0.13097882270812988
Tout mon soutien à @ZemmourEric pour ce soir. Il est le seul à pouvoir restaurer et incarner la fonction présidentielle. Sécurité, immigration, économie, préférence nationale … rendons sa grandeur à la France ! 🇫🇷
#JeSoutiensZemmour #ZemmourVsMelenchon
1441085740335521803
handle_tweet executed in 0.0005576610565185547
has_free_space executed in 0.1290123462677002
@zemou_ @CiciBobby Il y a les memes personnes que derrière macron, hollande, sarko, chirac,etc… 
Le FN des années 80-99 combattait pas cette manne financière mais depuis 2002 et l’ejection du pere c’est devenu un parti aussi triste que les autres
1441085740222320640
handle_tweet executed in 0.0005877017974853516
has_free_space executed in 0.136430025100708
@CNEWS @PascalPraud Pascal Praud quesque je rigole avec vous

has_free_space executed in 0.15384697914123535
@JulienZell @Tenshirock99 @Ben_Masnieres Ahaha ça va être mortel.
1441085825362456590
handle_tweet executed in 0.0006704330444335938
has_free_space executed in 0.12963199615478516
@_PSHA J’le disais toute a l’heure. Vraiment c’est fatiguant
1441085825643474950
handle_tweet executed in 0.0006322860717773438
has_free_space executed in 0.1602649688720703
Soirée Monstres &amp; Cie https://t.co/NHyEJ2R0d0
1441085832538955800
handle_tweet executed in 0.000682830810546875
has_free_space executed in 0.19145488739013672
@franceculture Déjà les zémmouriens devraient apprendre l'histoire de France. Je vous invite juste à voir la liste des prénoms de ceux qui ont combattu pour la liberté en France : https://t.co/JEUumYsjsO
1441085836246716418
handle_tweet executed in 0.0006608963012695312
has_free_space executed in 0.16094708442687988
@LouisWitter l'horreur, quasiment le nazisme
1441085842177462272
handle_tweet executed in 0.0006260871887207031
has_fr

has_free_space executed in 0.16272759437561035
#JeSoutiensMelenchon ce soir. Je ne sais pas encore pour qui je voterai (ni si je voterai tout court en 2022...) mais j'espère qu'il va bien faire fermer sa bouche à l'autre résidu de fond de capote qu'est Zemmour 🤮🤮🤮
1441085939359502346
handle_tweet executed in 0.0004887580871582031
has_free_space executed in 0.1667935848236084
@GuiraudInd Zemmour est raciste. Par analogie, ses électeurs le sont aussi. C'est tout. Qu'en est-il de la France alors ?
1441085944610754564
handle_tweet executed in 0.0007064342498779297
has_free_space executed in 0.16970324516296387
@_dieuoff Ça s'annonce légendaire https://t.co/O89ro9ChX6
1441085934577991682
downloading media
handle_tweet executed in 0.4265265464782715
has_free_space executed in 0.1481034755706787
@RaquelGarridoFr C'est l'heure de dire au revoir à Jean-Luc 👋
1441085944895991821
handle_tweet executed in 0.0005974769592285156
has_free_space executed in 0.12927985191345215
@FTarlet Quel succès
144

KeyError: 'data'